In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
from transformers import BertTokenizer, BertModel

c:\Users\Kaneki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

# Load the dataset (replace with your actual path)
df = pd.read_csv('data_cleaned_and_fixed.csv')

# Clean data: Drop rows with missing 'Tier 1' or 'Tier 2' or any essential columns
df = df.dropna(subset=['Tier 1', 'Tier 2', 'Title', 'Content', 'Description'])

# Combine the text fields for prediction
df['combined_text'] = df['Tier 2'] + ' ' + df['Title'] + ' ' + df['Content'] + ' ' + df['Description']


In [4]:
df

,Tier 1,Tier 2,URL,Title,Description,Keywords,Site Name,Content,License,Language,Tier 3,Tier 4,combined_text
0,Automotive,dash cam videos,https://www.techradar.com/best/best-dash-cam,The best dash cam 2024: The latest car cameras...,"Whatever your needs and budget, we've round up...",NaN,TechRadar,best dash cam nextbase garmin thinkware purcha...,cc by sa,en,NaN,NaN,dash cam videos The best dash cam 2024: The la...
1,Automotive,dash cam videos,https://www.amazon.com/Camera-Built-170%C2%B0W...,"Amazon.com: Dash Camera for Cars,4K Full UHD C...","Amazon.com: Dash Camera for Cars,4K Full UHD C...",NaN,Unknown Site,offer easy convenient return least one free re...,cc by sa,en,NaN,NaN,dash cam videos Amazon.com: Dash Camera for Ca...
2,Automotive,dash cam videos,https://matrackinc.com/dash-cams-with-cloud-st...,Top 6 Dash cams with Local and Cloud Storage -...,est dash cams with local and cloud storage for...,NaN,Matrackinc -,wed love send info fill form query one team me...,cc by sa,en,NaN,NaN,dash cam videos Top 6 Dash cams with Local and...
3,Automotive,dash cam videos,https://www.blackboxmycar.com/pages/license-pl...,License Plates in Dash Cam Footage — BlackboxM...,One of the reasons why you need a dash cam is ...,NaN,BlackboxMyCar,best dash cam best dash cam road incident inev...,cc by sa,en,NaN,NaN,dash cam videos License Plates in Dash Cam Foo...
4,Automotive,dash cam videos,https://www.hikvision.com/en/products/onboard-...,Industry Dash Camera - Dash Cameras - Hikvision,Hikvision dashcams provide a lightweight solut...,dash cameras,Hikvision,login global en product network product networ...,cc by sa,en,NaN,NaN,dash cam videos Industry Dash Camera - Dash Ca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33508,Automotive,scooters,https://www.bikewale.com/new-bike-search/best-...,"Best scooty under 60,000 | Top scooters under ...","Looking to buy a scooty under 60,000? Here is ...",NaN,BikeWale,rating parameter design styling reliability co...,cc,en,NaN,NaN,"scooters Best scooty under 60,000 | Top scoote..."
33509,Automotive,scooters,https://www.expat.com/forum/viewtopic.php?id=9...,Motor Scooter question - Mexico forum - Expat.com,I my name is Kirk I am a US citizen living in ...,NaN,https://www.expat.com/,motor scooter question last activity april chi...,cc,en,NaN,NaN,scooters Motor Scooter question - Mexico forum...
33510,Automotive,scooters,https://www.topgearpowersports.com/Inventory/A...,Scooters For Sale: Inventory at Top Gear Power...,Shop all in-stock Scooters inventory for sale ...,"inc, roselle il, top gear powersports, all inv...",Unknown Site,shop inventory sale dealership roselle illinoi...,cc,en,NaN,NaN,scooters Scooters For Sale: Inventory at Top G...
33511,Automotive,scooters,https://www.zigwheels.com/ola-electric-scooters,Ola Electric Scooters and Scooty Prices - New ...,Ola scooters offers 3 Electric models in price...,NaN,ZigWheels.com,ola electric scooter price start r x go upto r...,cc,en,NaN,NaN,scooters Ola Electric Scooters and Scooty Pric...


In [5]:
df = df.sample(frac=1)

In [6]:
filtered_df = df[['Tier 1', 'combined_text']]

In [7]:
filtered_df

,Tier 1,combined_text
16528,Medical Health,diseases and conditions Boiler Alert - How Doe...
882,Business and Finance,industries Open Licenses | resources.data.gov ...
32313,Automotive,auto technology Davis Technical College (Forme...
9953,Genres,talk radio List of progressive talk radio stat...
20859,Pets,reptiles Snake and Reptile Shack - Texas Disco...
...,...,...
32816,Automotive,auto type Self-driving cars: your complete gui...
602,Business and Finance,industries Oakton College: Connect to Degrees ...
28690,Travel,travel preparation and advice Travel Planning ...
16615,Medical Health,diseases and conditions Chart: Where Diabetes ...


In [8]:
! pip install datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached pyarrow-18.1.0-cp311-cp311-win_amd64.whl (25.1 MB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl (30 kB)
  Attempting unin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seleniumbase 4.31.4 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [40]:
import torch.nn.functional as F


In [ ]:
len(filtered_df['label'])

In [9]:

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
from torch import nn
from transformers import BertModel

# Map Tier 1 labels to integers
label_mapping = {label: idx for idx, label in enumerate(filtered_df['Tier 1'].unique())}
filtered_df['label'] = filtered_df['Tier 1'].map(label_mapping)

# Step 2: Split Data
train_df, val_df = train_test_split(filtered_df, test_size=0.2, random_state=42)

# Step 3: Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def preprocess_function(examples):
    return tokenizer(examples['combined_text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\323329675.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = filtered_df['Tier 1'].map(label_mapping)
Map: 100%|██████████| 6139/6139 [01:43<00:00, 59.08 examples/s]


In [10]:
len(train_df)


24555

In [89]:
number_of_classes=filtered_df['label'].unique()

In [90]:
number_of_classes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=int64)

In [13]:
number_of_classes

31

In [15]:
class CustomBERTModel(nn.Module):
    def __init__(self, num_labels):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.hidden_layer = nn.Linear(self.bert.config.hidden_size, 100)  # Hidden layer output size is 5
        self.classifier = nn.Linear(100, num_labels)  # Output layer should match the number of classes (22 in your case)

    def forward(self, input_ids, attention_mask, labels=None):
        print("forward")
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        hidden_output = torch.relu(self.hidden_layer(pooled_output))
        logits = self.classifier(hidden_output)
        print(logits.shape)
        # Calculate softmax probabilities
        softmax_probs = F.softmax(logits, dim=-1)

        # Get top 5 classes and their softmax probabilities
        top5_probs, top5_classes = torch.topk(softmax_probs, 5, dim=-1)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return {
            "loss": loss,
            "logits": logits,
            "softmax_probs": softmax_probs,
            "top5_probs": top5_probs,
            "top5_classes": top5_classes,
        }

In [16]:

# Set format for PyTorch
train_dataset = train_dataset.rename_column("label", "labels").with_format("torch")
val_dataset = val_dataset.rename_column("label", "labels").with_format("torch")

In [17]:
import torch
import torch.nn.functional as F
import numpy as np

# Step 6: Define Metrics
def compute_metrics(pred):
    logits, labels = pred

    # Convert logits to PyTorch tensor
    logits = torch.tensor(logits)
    
    # Convert labels to PyTorch tensor if they are numpy arrays
    if isinstance(labels, np.ndarray):
        labels = torch.tensor(labels)
    
    # Calculate softmax probabilities
    softmax_probs = F.softmax(logits, dim=-1)  # Get softmax probabilities
    
    # Get top 5 classes and their softmax probabilities
    top5_probs, top5_classes = torch.topk(softmax_probs, 5, dim=-1)
    
    # Use unsqueeze to add a dimension to the labels tensor to compare it with top5_classes
    labels = labels.unsqueeze(1)  # Make sure labels are of shape (batch_size, 1)

    # Check if the true label is among the top 5 predictions
    correct = (top5_classes == labels).sum().item()  # Check if true label is in top 5
    top5_accuracy = correct / labels.size(0)  # Compute top 5 accuracy

    return {"top5_accuracy": top5_accuracy}

In [18]:
train_dataset

Dataset({
    features: ['Tier 1', 'combined_text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 24555
})

In [52]:
from transformers import BertTokenizer, Trainer, TrainingArguments
from sklearn.metrics import top_k_accuracy_score

In [19]:
# Initialize Model
# num_labels = 26  # Number of labels for classification (26 classes)
model = CustomBERTModel(num_labels=number_of_classes)

# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define custom compute_metrics function for top-5 accuracy
# def compute_metrics(p):
#     predictions, labels = p
#     # Get top-5 accuracy
#     top5_acc = top_k_accuracy_score(labels, predictions, k=5)
#     return {
#         "eval_top5_accuracy": top5_acc
#     }

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Use top-5 accuracy for evaluation
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,  # Pass the compute_metrics function
)

# Train the Model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_custom_bert")

c:\Users\Kaneki\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\3572779594.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/4605 [00:00<?, ?it/s]

forward
torch.Size([16, 31])


  0%|          | 1/4605 [00:01<1:27:38,  1.14s/it]

forward
torch.Size([16, 31])


  0%|          | 2/4605 [00:01<1:13:51,  1.04it/s]

...

forward
torch.Size([11, 31])


                                                     
100%|██████████| 4605/4605 [1:13:29<00:00,  1.24it/s]

{'eval_loss': 0.002857289044186473, 'eval_runtime': 105.6435, 'eval_samples_per_second': 58.111, 'eval_steps_per_second': 3.635, 'epoch': 3.0}


100%|██████████| 4605/4605 [1:13:36<00:00,  1.04it/s]


{'train_runtime': 4416.398, 'train_samples_per_second': 16.68, 'train_steps_per_second': 1.043, 'train_loss': 0.16826542917473178, 'epoch': 3.0}


In [20]:
from torch.utils.data import DataLoader


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
from torch.utils.data import DataLoader


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [24]:
device

device(type='cuda')

In [25]:
model.to(device)

CustomBERTModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [26]:
val_dataset

Dataset({
    features: ['Tier 1', 'combined_text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6139
})

In [40]:
top_five_classes = []

In [41]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from datasets import Dataset

# Step 1: Detect the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming the trained model is loaded

model.to(device)  # Move model to the detected device
model.eval()  # Set the model to evaluation mode

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Step 2: Tokenization for the validation dataset (if not already tokenized)
def preprocess_function(examples):
    return tokenizer(examples['combined_text'], padding="max_length", truncation=True)

val_dataset = val_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.with_format("torch")

# Step 3: Create DataLoader for the validation set
val_loader = DataLoader(val_dataset, batch_size=16)

# Step 4: Make Predictions
predicted_labels = []

with torch.no_grad():  # No need to calculate gradients for prediction
    for batch in val_loader:
        # Move data to the same device as the model
        input_ids = batch['input_ids'].to(device)  # Move input_ids to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention_mask to device
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs['logits']

        print(outputs['top5_classes'])

        print(outputs['top5_probs'])
        top_five_classes.append(outputs['top5_classes'])
#         # Get predicted class (index of the max logit)
#         predictions = torch.argmax(logits, dim=1)
        
#         # Store the predicted labels
#         predicted_labels.extend(predictions.tolist())

# # Step 5: Convert Predictions to Corresponding Labels (if you have the label mapping)
# reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}
# predicted_labels_named = [reverse_label_mapping[label] for label in predicted_labels]

# # Print or return the list of predicted labels
# print(predicted_labels_named)


Map: 100%|██████████| 6139/6139 [01:56<00:00, 52.82 examples/s]


forward
torch.Size([16, 31])
tensor([[ 2, 16,  0, 12, 19],
        [19, 18,  2,  6, 25],
        [26, 28, 29, 25, 17],
        [ 0, 29, 28,  8,  4],
        [ 3, 17, 23, 14, 10],
        [19, 18, 10,  2, 25],
        [20, 14, 16, 18, 12],
        [ 5, 30, 19, 24, 29],
        [13,  1, 16,  7, 24],
        [ 0, 29,  8, 28,  4],
        [ 2, 16,  0, 12, 19],
        [19, 18,  2,  6, 10],
        [ 3, 17, 23, 14, 10],
        [13,  1,  7, 24, 16],
        [13,  1, 16,  7, 24],
        [ 4,  0,  8, 28, 21]], device='cuda:0')
tensor([[9.9966e-01, 5.2339e-05, 3.9383e-05, 3.8346e-05, 3.7063e-05],
        [9.9963e-01, 7.2007e-05, 3.8744e-05, 3.5616e-05, 3.5045e-05],
        [9.9898e-01, 1.8509e-04, 1.7532e-04, 1.2903e-04, 1.0593e-04],
        [9.9964e-01, 1.0301e-04, 4.6391e-05, 4.5927e-05, 4.3011e-05],
        [9.9939e-01, 1.2828e-04, 8.8389e-05, 8.2604e-05, 7.5756e-05],
        [9.9963e-01, 7.1241e-05, 3.7505e-05, 3.7271e-05, 3.6325e-05],
        [9.9684e-01, 5.8166e-04, 4.4227e-04, 2.7593e-

In [82]:
len(val_loader)

384

In [29]:
val_df

,Tier 1,combined_text,label
31140,Automotive,auto type Strategy Study: How Ferrari Became T...,2
14461,Hobbies & Interests,"arts and crafts 4,400+ Best Digital Scrapbook ...",19
7930,Food & Drink,alcoholic beverages 28 Popular Store-Bought Ca...,26
15252,Medical Health,diseases and conditions Carolina Vision Care -...,0
8828,Genres,"soap opera 93,219 Soap Opera Photos & High Res...",3
...,...,...,...
21461,Personal Finance,home utilities Different types of internet ser...,13
25390,Style & Fashion,women's fashion Marc Jacobs | Official Site si...,25
26928,Technology & Computing,consumer electronics Statewide Dual Credit Pri...,5
6375,Family and Relationships,dating Licensing Fees - California DMV find fe...,24


In [ ]:
out = []

In [70]:
final_out = []

In [78]:
len(out)

768

In [64]:
for pre in top_five_classes:
    out.append(pre.cpu())

In [71]:
for o in out:
    for i in o:
        final_out.append(i.cpu())

In [72]:
final_out

[tensor([ 2, 16,  0, 12, 19]),
 tensor([19, 18,  2,  6, 25]),
 tensor([26, 28, 29, 25, 17]),
 tensor([ 0, 29, 28,  8,  4]),
 tensor([ 3, 17, 23, 14, 10]),
 tensor([19, 18, 10,  2, 25]),
 tensor([20, 14, 16, 18, 12]),
 tensor([ 5, 30, 19, 24, 29]),
 tensor([13,  1, 16,  7, 24]),
 tensor([ 0, 29,  8, 28,  4]),
 tensor([ 2, 16,  0, 12, 19]),
 tensor([19, 18,  2,  6, 10]),
 tensor([ 3, 17, 23, 14, 10]),
 tensor([13,  1,  7, 24, 16]),
 tensor([13,  1, 16,  7, 24]),
 tensor([ 4,  0,  8, 28, 21]),
 tensor([19, 18,  2, 10, 25]),
 tensor([ 0, 29,  8, 28,  4]),
 tensor([21, 16,  8, 27,  7]),
 tensor([15, 27, 10, 21, 14]),
 tensor([ 3, 17, 23, 14, 10]),
 tensor([23,  3, 19, 30, 11]),
 tensor([24,  8, 25, 22, 13]),
 tensor([28,  4, 14, 26,  0]),
 tensor([ 2, 16,  0, 12, 11]),
 tensor([10, 29, 19, 27, 15]),
 tensor([12, 28, 30, 19,  1]),
 tensor([19, 18,  2, 10,  6]),
 tensor([ 3, 17, 23, 14, 10]),
 tensor([ 2, 16,  0, 12, 19]),
 tensor([25, 24, 19,  6,  9]),
 tensor([13,  1,  7, 16, 24]),
 tensor(

In [83]:
len(final_out)

12278

In [86]:
top_five_classes = np.array(final_out[:6139])

In [87]:
top_five_classes.shape

(6139, 5)

In [36]:
outputs['top5_probs']

tensor([[9.9731e-01, 7.1162e-04, 4.3581e-04, 4.2286e-04, 2.1610e-04],
        [9.9961e-01, 1.1752e-04, 6.4840e-05, 3.2382e-05, 3.0732e-05],
        [9.9896e-01, 1.6974e-04, 1.4532e-04, 1.3028e-04, 1.1010e-04],
        [9.9896e-01, 1.8019e-04, 1.7970e-04, 1.3812e-04, 1.0380e-04],
        [9.9930e-01, 9.2277e-05, 8.2249e-05, 7.7140e-05, 7.4389e-05],
        [9.9975e-01, 6.5355e-05, 2.7687e-05, 2.0069e-05, 1.8918e-05],
        [9.9973e-01, 9.5657e-05, 2.8310e-05, 2.6133e-05, 2.5357e-05],
        [9.9920e-01, 1.3118e-04, 1.2397e-04, 1.1303e-04, 7.5279e-05],
        [9.9974e-01, 6.5676e-05, 2.8522e-05, 2.0640e-05, 1.8611e-05],
        [9.9929e-01, 9.5041e-05, 8.4232e-05, 7.8581e-05, 7.3244e-05],
        [9.9955e-01, 8.6234e-05, 5.2170e-05, 3.8804e-05, 3.6203e-05]],
       device='cuda:0')

In [50]:
out = []

In [ ]:
val_df.head()

,Tier 1,combined_text,label
31140,Automotive,auto type Strategy Study: How Ferrari Became T...,2
14461,Hobbies & Interests,"arts and crafts 4,400+ Best Digital Scrapbook ...",19
7930,Food & Drink,alcoholic beverages 28 Popular Store-Bought Ca...,26
15252,Medical Health,diseases and conditions Carolina Vision Care -...,0
8828,Genres,"soap opera 93,219 Soap Opera Photos & High Res...",3


In [115]:
val_df.iloc[0].name

31140

In [76]:
len(val_df)

6139

In [116]:
# Load the dataset (replace with your actual path)
copy_df = pd.read_csv('data_cleaned_and_fixed.csv')

# Clean data: Drop rows with missing 'Tier 1' or 'Tier 2' or any essential columns
copy_df = copy_df.dropna(subset=['Tier 1', 'Tier 2', 'Title', 'Content', 'Description'])

In [117]:
copy_df

,Tier 1,Tier 2,URL,Title,Description,Keywords,Site Name,Content,License,Language,Tier 3,Tier 4
0,Automotive,dash cam videos,https://www.techradar.com/best/best-dash-cam,The best dash cam 2024: The latest car cameras...,"Whatever your needs and budget, we've round up...",NaN,TechRadar,best dash cam nextbase garmin thinkware purcha...,cc by sa,en,NaN,NaN
1,Automotive,dash cam videos,https://www.amazon.com/Camera-Built-170%C2%B0W...,"Amazon.com: Dash Camera for Cars,4K Full UHD C...","Amazon.com: Dash Camera for Cars,4K Full UHD C...",NaN,Unknown Site,offer easy convenient return least one free re...,cc by sa,en,NaN,NaN
2,Automotive,dash cam videos,https://matrackinc.com/dash-cams-with-cloud-st...,Top 6 Dash cams with Local and Cloud Storage -...,est dash cams with local and cloud storage for...,NaN,Matrackinc -,wed love send info fill form query one team me...,cc by sa,en,NaN,NaN
3,Automotive,dash cam videos,https://www.blackboxmycar.com/pages/license-pl...,License Plates in Dash Cam Footage — BlackboxM...,One of the reasons why you need a dash cam is ...,NaN,BlackboxMyCar,best dash cam best dash cam road incident inev...,cc by sa,en,NaN,NaN
4,Automotive,dash cam videos,https://www.hikvision.com/en/products/onboard-...,Industry Dash Camera - Dash Cameras - Hikvision,Hikvision dashcams provide a lightweight solut...,dash cameras,Hikvision,login global en product network product networ...,cc by sa,en,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
33508,Automotive,scooters,https://www.bikewale.com/new-bike-search/best-...,"Best scooty under 60,000 | Top scooters under ...","Looking to buy a scooty under 60,000? Here is ...",NaN,BikeWale,rating parameter design styling reliability co...,cc,en,NaN,NaN
33509,Automotive,scooters,https://www.expat.com/forum/viewtopic.php?id=9...,Motor Scooter question - Mexico forum - Expat.com,I my name is Kirk I am a US citizen living in ...,NaN,https://www.expat.com/,motor scooter question last activity april chi...,cc,en,NaN,NaN
33510,Automotive,scooters,https://www.topgearpowersports.com/Inventory/A...,Scooters For Sale: Inventory at Top Gear Power...,Shop all in-stock Scooters inventory for sale ...,"inc, roselle il, top gear powersports, all inv...",Unknown Site,shop inventory sale dealership roselle illinoi...,cc,en,NaN,NaN
33511,Automotive,scooters,https://www.zigwheels.com/ola-electric-scooters,Ola Electric Scooters and Scooty Prices - New ...,Ola scooters offers 3 Electric models in price...,NaN,ZigWheels.com,ola electric scooter price start r x go upto r...,cc,en,NaN,NaN


In [119]:
# Create a mapping from integer value to text label
int_to_label = {row['label']: row['Tier 1'] for _, row in val_df.iterrows()}

# Iterate over the data and print true and predicted labels
for i, top_classes in enumerate(top_five_classes):
    # Get the true text label for this sample using iloc
    # true_label = val_df.iloc[i, 0]  # 0 is the column index for 'Tier 1'
    actual_index = val_df.iloc[i].name  # Retrieve the index of the current row
    
    # Use the index to get the Tier 2 value from the original_df
    true_tier_2_label = copy_df.loc[actual_index, 'Tier 2']
    
    # Map predicted indices to text labels
    predicted_labels = [int_to_label[idx] for idx in top_classes]

    # Print results
    print(f"Sample {i + 1}:")
    print(f"  True Label: {true_tier_2_label}")
    print(f"  Predicted Labels: {predicted_labels}")

Sample 1:
  True Label: auto type
  Predicted Labels: ['Automotive', 'Science', 'Medical Health', 'Travel', 'Hobbies & Interests']
Sample 2:
  True Label: arts and crafts
  Predicted Labels: ['Hobbies & Interests', 'Pop Culture', 'Automotive', 'Fine Art', 'Style & Fashion']
Sample 3:
  True Label: alcoholic beverages
  Predicted Labels: ['Food & Drink', 'Attractions', 'Shopping', 'Style & Fashion', 'Personal Celebrations & Life Events']
Sample 4:
  True Label: diseases and conditions
  Predicted Labels: ['Medical Health', 'Shopping', 'Attractions', 'Healthy Living', 'Pets']
Sample 5:
  True Label: soap opera
  Predicted Labels: ['Genres', 'Personal Celebrations & Life Events', 'Careers', 'Events', 'Video Gaming']
Sample 6:
  True Label: content production
  Predicted Labels: ['Hobbies & Interests', 'Pop Culture', 'Video Gaming', 'Automotive', 'Style & Fashion']
Sample 7:
  True Label: elections
  Predicted Labels: ['Politics', 'Events', 'Science', 'Pop Culture', 'Travel']
Sample 8:
  T

In [127]:
from neo4j import GraphDatabase

class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def create_node_and_relationships(self, true_label, predicted_labels):
        with self.driver.session() as session:
            session.write_transaction(self._create_relationships, true_label, predicted_labels)
    
    @staticmethod
    def _create_relationships(tx, true_label, predicted_labels):
        # Add visual differences through properties (e.g., color, shape)
        query = """
        MERGE (true_label_node:Label {name: $true_label, type: 'true', color: 'blue', shape: 'circle'})
        WITH true_label_node
        UNWIND $predicted_labels AS predicted
        MERGE (predicted_node:Label {name: predicted, type: 'predicted', color: 'red', shape: 'square'})
        MERGE (true_label_node)-[:MATCHES]->(predicted_node)
        """
        tx.run(query, true_label=true_label, predicted_labels=predicted_labels)

# Initialize connection
kg = KnowledgeGraph("bolt://localhost:7687", "neo4j", "kenkaneki")

In [ ]:
top_five_classes[0]

array([[ 2, 16,  0, 12, 19],
       [19, 18,  2,  6, 25],
       [26, 28, 29, 25, 17],
       ...,
       [ 5, 30, 19, 24, 29],
       [24,  8, 25, 22, 13],
       [11,  3,  7, 23, 18]], dtype=int64)

In [149]:
i = 26
item = top_five_classes[26]

In [150]:
# true_label = val_df.iloc[i, 0]  # 0 is the column index for 'Tier 1'
actual_index = val_df.iloc[i].name  # Retrieve the index of the current row

# Use the index to get the Tier 2 value from the original_df
true_tier_2_label = copy_df.loc[actual_index, 'Tier 2']

true_tier_1_label = copy_df.loc[actual_index, 'Tier 1']

# Map predicted indices to text labels
predicted_labels = [int_to_label[idx] for idx in item]

# Add to Neo4j
# kg.create_node_and_relationships(true_tier_2_label, predicted_labels)

# # Close the connection when done
# kg.close()



print(true_tier_2_label, true_tier_1_label, predicted_labels)

travel type Travel ['Travel', 'Attractions', 'Holidays', 'Hobbies & Interests', 'Business and Finance']


In [153]:
import random
from neo4j import GraphDatabase

# Connect to the Neo4j database
uri = "bolt://localhost:7687"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your username
password = "kenkaneki"  # Replace with your password

driver = GraphDatabase.driver(uri, auth=(username, password))

def create_graph(true_tier_2_label, true_tier_1_label, predicted_labels):
    with driver.session() as session:
        # Create the central node for true_tier_2_label with blue color
        session.write_transaction(create_true_tier_2_node, true_tier_2_label)

        # Create nodes for true_tier_1_label and predicted_labels
        for label in predicted_labels:
            # Assign random colors to predicted labels
            color = random_color() if label != true_tier_1_label else "red"
            session.write_transaction(create_predicted_label_node, label, color)
        
        # Create relationships between true_tier_2_label and predicted labels
        for label in predicted_labels:
            session.write_transaction(create_relationship, true_tier_2_label, label)
        
        # Create "ACTUAL_LABEL" relationship for the matching true_tier_1_label
        if true_tier_1_label in predicted_labels:
            session.write_transaction(create_actual_label_relationship, true_tier_2_label, true_tier_1_label)

def create_true_tier_2_node(tx, true_tier_2_label):
    query = f"""
    MERGE (t2:`{true_tier_2_label}` {{name: '{true_tier_2_label}', color: 'blue'}})
    """
    tx.run(query)

def create_predicted_label_node(tx, label, color):
    query = f"""
    MERGE (p:`{label}` {{name: '{label}', color: '{color}'}})
    """
    tx.run(query)

def create_relationship(tx, true_tier_2_label, predicted_label):
    query = f"""
    MATCH (t2:`{true_tier_2_label}` {{name: '{true_tier_2_label}'}})
    MATCH (p:`{predicted_label}` {{name: '{predicted_label}'}})
    MERGE (t2)-[:MATCHED]->(p)
    """
    tx.run(query)

def create_actual_label_relationship(tx, true_tier_2_label, true_tier_1_label):
    query = f"""
    MATCH (t2:`{true_tier_2_label}` {{name: '{true_tier_2_label}'}})
    MATCH (p:`{true_tier_1_label}` {{name: '{true_tier_1_label}'}})
    MERGE (t2)-[:ACTUAL_LABEL]->(p)
    """
    tx.run(query)

def random_color():
    # Return a random color in hex format
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))


# Create the graph
# create_graph(true_tier_2_label, true_tier_1_label, predicted_labels)

# Close the connection
# driver.close()

In [155]:
# Iterate over the data and print true and predicted labels
for i, top_classes in enumerate(top_five_classes[0:10]):
    # Get the true text label for this sample using iloc
    # true_label = val_df.iloc[i, 0]  # 0 is the column index for 'Tier 1'
    actual_index = val_df.iloc[i].name  # Retrieve the index of the current row
    
    # Use the index to get the Tier 2 value from the original_df
    true_tier_2_label = copy_df.loc[actual_index, 'Tier 2']

    true_tier_1_label = copy_df.loc[actual_index, 'Tier 1']
    
    # Map predicted indices to text labels
    predicted_labels = [int_to_label[idx] for idx in top_classes]

    # Add to Neo4j
    # kg.create_node_and_relationships(true_tier_2_label, predicted_labels)
    # Create the graph
    create_graph(true_tier_2_label, true_tier_1_label, predicted_labels)

    # # Close the connection when done
    driver.close()

    print(true_tier_2_label, true_tier_1_label, predicted_labels)

C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\2082245439.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\2082245439.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_true_tier_2_node, true_tier_2_label)
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\2082245439.py:20: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_predicted_label_node, label, color)
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\2082245439.py:24: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, true_tier_2_label, label)
C:\Users\Kaneki\AppData\Local\Temp\ipykernel_40276\2082245439.py:28: DeprecationWarning: write_transaction has been renamed to e

auto type Automotive ['Automotive', 'Science', 'Medical Health', 'Travel', 'Hobbies & Interests']
arts and crafts Hobbies & Interests ['Hobbies & Interests', 'Pop Culture', 'Automotive', 'Fine Art', 'Style & Fashion']
alcoholic beverages Food & Drink ['Food & Drink', 'Attractions', 'Shopping', 'Style & Fashion', 'Personal Celebrations & Life Events']
diseases and conditions Medical Health ['Medical Health', 'Shopping', 'Attractions', 'Healthy Living', 'Pets']
soap opera Genres ['Genres', 'Personal Celebrations & Life Events', 'Careers', 'Events', 'Video Gaming']
content production Hobbies & Interests ['Hobbies & Interests', 'Pop Culture', 'Video Gaming', 'Automotive', 'Style & Fashion']
elections Politics ['Politics', 'Events', 'Science', 'Pop Culture', 'Travel']
virtual reality Technology & Computing ['Technology & Computing', 'Holidays', 'Hobbies & Interests', 'Family and Relationships', 'Shopping']
personal debt Personal Finance ['Personal Finance', 'Business and Finance', 'Science'